## Instituto Superior de Engenharia de Lisboa 
## LEIM - 2024/2025
### PIV - Processamento de Imagem e Visão

# <br>

#### <center> Trabalho Prático 2.A - Estimação e Classificação de Movimento </center>
<b> <center> Grupo 14 </center> </b> 
# <br>



Trabalho realizado por:

* João Fonseca nº<b> 49707 </b>
* Diogo Coito nº<b> 50029 </b>

 Docente: Pedro Mendes Jorge 
# <br>

## Introdução

Neste segundo trabalho prático, foi-nos pedido para desenvolver um algoritmo de estimação e classificação de movimento. Para isso, dividimos o processo de elaboração deste em 3 partes:

- Deteção da região da mão 
- Classificação dos movimentos
- Fazer a representação gráfica

Cada um destes passos vai ser abordado de forma mais detalhada no desenvolvimento do relatório, bem como o código produzido de maneira a atingir o objetivo

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

## Desenvolvimento

#### 1. Deteção e seguimento da região da mão

Neste primeiro passo, procurámos isolar a mão do resto do conteúdo que a câmera capta, de maneira a obter os contornos desta. Para isso, decidimos que a melhor maneira seria através da deteção do tom de pele. Como o tom de pele pode variar de acordo com a intensidade da luz e a direção desta, começámos por converter a imagem recebida para HSV por este formato nos dar mais controlo sobre a luminosidade e saturação. Seguidamente, definimos valores mínimos e máximos de maneira a fazer a binarização da imagem e fazer a extração dos seus contornos. Após obtermos os contornos, é preciso definir aquelas que pertencem à mão. Para isso, definimos algumas condições que fazem com que contornos com área inferior a 5000 não sejam tidos em conta e que o contorno com a coordenada y mais baixa seja identificado como sendo a mão

In [2]:
def detetaMao(img):

    # conversão da image de RGB para HSV. Neste caso, o HSV é mais vantajoso relativamente ao RGB por permiter definir a saturação e o brilho
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # limites definidos para detetar o tom de pele
    limiar_baixo = np.array([0, 40, 50])
    limiar_cima = np.array([20, 150, 255])
    
    nova_img = cv2.inRange(img_hsv, limiar_baixo, limiar_cima) # binarização da imagem
    
    # operadores morfológicos de abertura e fecho
    op = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    nova_img = cv2.morphologyEx(nova_img, cv2.MORPH_CLOSE, op)
    nova_img = cv2.morphologyEx(nova_img, cv2.MORPH_OPEN, op)
    
    contornos, _ = cv2.findContours(nova_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contorno_malto = None
    for contorno in contornos:
        area = cv2.contourArea(contorno)
        if area > 5000: 
            x, y, larg, alt = cv2.boundingRect(contorno)
            if not contorno_malto or y < contorno_malto['y']:
                contorno_malto = {'x': x, 'y': y, 'largura': larg, 'altura': alt, 'area': area}
    
    return contorno_malto

#### 2. Classificação dos movimentos

Tendo o contorno que define a mão identificado, precisámos de identificar os tipos de movimentos realizados por esta. Estes podiam ser ESQUERDA, DIREITA, CIMA, BAIXO, ZOOM IN e ZOOM OUT. Para isso fizémos uma análise das coordenas x e y, bem como da área dos contornos em imagens consecutivas para fazer essa classificação.

A partir dos nossos testes, concluímos que a mão não está sempre estável do ponto de vista computacional e para ter uma identificação de movimentos fidedigna precisávamos de criar uma função que não tivesse em conta pequenas alterações nas coordenadas. Por isso, criámos uma função de suavização das coordenadas.

A classificação de movimentos é feita da seguinte maneira: são verificadas as coordenadas e área da posição atual e da posição anterior, estando estas guardadas num array. É calculada a variação de coordenas e área entre as duas posições e com base nisso o movimento é classificado. As condições para a sua classificação são as seguintes:
- Se a variação das coordenadas e da área for inferior aos limiares definidos, é classificado como não havendo movimento
- Se a variação da área for superior ao limiar de área definido, é classificado como ZOOM IN
- Se a variação da área for inferior ao valor negativo do limiar de área definido, é classificado como ZOOM OUT
- Se a variação no eixo do x for superior à variação do eixo do y e a variação de x tiver um valor positivo, é classificado como DIREITA
- Se a variação no eixo do x for superior à variação do eixo do y e a variação de x tiver um valor negativo, é classificado como ESQUERDA
- Se a variação no eixo do y for superior à variação do eixo do x e a variação de y tiver um valor positivo, é classificado como BAIXO
- Se a variação no eixo do y for superior à variação do eixo do x e a variação de y tiver um valor negativo, é classificado como CIMA

In [7]:
# função para suavizar os movimentos
def smoothValues(current_value, previous_value, smoothing_factor=0.8):
    return smoothing_factor*previous_value+(1-smoothing_factor)*current_value

# função de classificação dos movimentos
def classifyMovement(posicoes, thresholds):
    
    if len(posicoes) < 2:
        return "Sem Movimento"

    # vai buscar os dois últimos movimentos registados
    anterior = posicoes[-2]
    atual = posicoes[-1]
    
    dx = atual['x'] - anterior['x']
    dy = atual['y'] - anterior['y']
    d_area = atual['area'] - anterior['area']

    if abs(dx) < thresholds['dx'] and abs(dy) < thresholds['dy'] and abs(d_area) < thresholds['d_area']:# filtra pequenas variações 
        return "Sem Movimento"

    # classificar com base nas mudanças de coordendas ou área
    if d_area > thresholds['d_area']:  
        return "Zoom In"
        
    elif d_area < -thresholds['d_area']: 
        return "Zoom Out"
        
    elif abs(dx) > abs(dy):  
        return "Direita" if dx > 0 else "Esquerda"
        
    elif abs(dy) > abs(dx):
        return "Baixo" if dy > 0 else "Cima"
    
    return "Sem movimento"




import cv2
import numpy as np

def detetaMao(img):
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    limiar_baixo = np.array([0, 40, 50])
    limiar_cima = np.array([20, 150, 255])

    nova_img = cv2.inRange(img_hsv, limiar_baixo, limiar_cima)

    op = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    nova_img = cv2.morphologyEx(nova_img, cv2.MORPH_CLOSE, op)
    nova_img = cv2.morphologyEx(nova_img, cv2.MORPH_OPEN, op)

    contornos, _ = cv2.findContours(nova_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contorno_malto = None

    for contorno in contornos:
        area = cv2.contourArea(contorno)
        if area > 5000:
            x, y, larg, alt = cv2.boundingRect(contorno)
            if not contorno_malto or y < contorno_malto['y']:
                contorno_malto = {'x': x, 'y': y, 'largura': larg, 'altura': alt, 'area': area}

    return contorno_malto

def smoothValues(current_value, previous_value, smoothing_factor=0.8):
    return smoothing_factor * previous_value + (1 - smoothing_factor) * current_value

def classifyMovement(posicoes, thresholds):
    if len(posicoes) < 2:
        return "Sem Movimento"

    anterior = posicoes[-2]
    atual = posicoes[-1]

    dx = atual['x'] - anterior['x']
    dy = atual['y'] - anterior['y']
    d_area = atual['area'] - anterior['area']

    if abs(dx) < thresholds['dx'] and abs(dy) < thresholds['dy'] and abs(d_area) < thresholds['d_area']:
        return "Sem Movimento"

    if d_area > thresholds['d_area']:
        return "Zoom In"
    elif d_area < -thresholds['d_area']:
        return "Zoom Out"
    elif abs(dx) > abs(dy):
        return "Direita" if dx > 0 else "Esquerda"
    elif abs(dy) > abs(dx):
        return "Baixo" if dy > 0 else "Cima"

    return "Sem Movimento"



'''
cap = cv2.VideoCapture(0)

posicoes = []
limiares = {'dx':20,'dy':20,'d_area':1500}  

while True:
    ret, frame = cap.read()    
    posicao_atual = detetaMao(frame)
    if posicao_atual:
        posicoes.append(posicao_atual)
        if len(posicoes) > 10:
            posicoes.pop(0)
        movimento = classifyMovement(posicoes, limiares)
        cv2.rectangle(frame, (posicao_atual['x'], posicao_atual['y']),
                      (posicao_atual['x'] + posicao_atual['largura'], posicao_atual['y'] + posicao_atual['altura']),
                      (0, 255, 0), 2)
        cv2.putText(frame, movimento, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow("Deteta mao", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

#### 3. Representação gráfica

Neste último passo, era apenas preciso fazer o desenho gráfico do objeto que queriámos e corresponder o seu movimento às classificações feitas do movimento da mão. Para isso construímos um triângulo, definindo um tamanho e as coordenadas centrais. Este tamanho e coordenadas vão ser guardados em variáveis globais de maneira a que sejam ajustados de acordo com os movimentos.
Quando um movimento é detetado e classificado, o triângulo ajusta a sua posição ou tamanho de acordo com este. Por exemplo, quando é detetado o movimento ESQUERDA, a coordenada x do centro do triângulo tem o seu valor decrementado em 20 unidades.

In [22]:
triangle_center = [300, 300] 
triangle_size = 50  


def draw_triangle(frame, center, size):
    
    vertices = np.array([ [center[0],center[1]-size],[center[0]-size,center[1]+size],[center[0]+size,center[1]+size]])
    cv2.polylines(frame, [vertices], isClosed=True, color=(0, 255, 0), thickness=3) # desenha o triângulo

    return frame

# atualização da posição do triângulo
def update_triangle(movement, center, size, move_step, zoom_step):
    
    if movement == "Direita":
        center[0] += move_step
    elif movement == "Esquerda":
        center[0] -= move_step
    elif movement == "Cima":
        center[1] -= move_step
    elif movement == "Baixo":
        center[1] += move_step
    elif movement == "Zoom In":
        size += zoom_step
    elif movement == "Zoom Out":
        size = max(size-zoom_step, 10) # evita que o triângulo diminua em excesso

    return center, size


def teste3():
    
    global triangle_center, triangle_size
    
    window_name = "Animação"
    frame = np.zeros((800,800,3))
    cap = cv2.VideoCapture(0)
    posicoes = []
    smoothed_position = {'x':0,'y':0,'area':0}  
    # Limiares de movimento
    limiares = {'dx':20,'dy':20,'d_area':1500}  
    posicoes = [] 
    move_step = 20 
    zoom_step = 50 
    cap = cv2.VideoCapture(0)

    while True:
        ret, img = cap.read()
        hand = detetaMao(img)
        
        if hand:
            # Ajustar os valores de maneira a que as ações não mudem rapidamente
            smoothed_position['x'] = smoothValues(hand['x'],smoothed_position['x'])
            smoothed_position['y'] = smoothValues(hand['y'],smoothed_position['y'])
            smoothed_position['area'] = smoothValues(hand['area'],smoothed_position['area'])
            posicoes.append(smoothed_position.copy())
            
            if len(posicoes) > 30:
                posicoes.pop(0)

            movement = classifyMovement(posicoes, limiares)# classificação do movimento
            triangle_center,triangle_size = update_triangle(movement,triangle_center,triangle_size,move_step,zoom_step) # atualização da 
                                                                                                                        # posição do triângulo 
        frame = np.zeros((800,800,3))
        frame = draw_triangle(frame,triangle_center,triangle_size)
        cv2.putText(frame,f"Movement: {movement}",(10, 50),cv2.FONT_HERSHEY_SIMPLEX,0.7,(255, 255, 255),2)
        cv2.imshow(window_name,frame)

        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break 

    cap.release()
    cv2.destroyAllWindows()

teste3()

## Conclusão

Em jeito de conclusão, o trabalho alcançou o seu objetivo final de estimar e classifar movimentos, exemplificando o seu bom funcionamento através de uma aplicação gráfica interativa. A biblioteca OpenCV revelou-se extremamente útil nesta trabalho, tendo sido utilizada na maioria dos passos do trabalho. No entanto, o bom funcionamento deste não consegue ser sempre garantido, pois se as condições de luminosidade e o conteúdo captado pela câmera não se assemelhar às condições de teste, podem afetar o resultado final.